In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
# school_data_complete = school_data_complete.sort_values(by="school_name")
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


District Summary

In [2]:
# total number of schools
total_schools = school_data['school_name'].count()

# total number of students
total_students = student_data['student_name'].count()
# student total will need to be formatted - str can't have calculations performed on them
total_students_formatted = "{:,}".format(total_students)

# total budget - help from kite.com
total_budget = "${:,.2f}".format(school_data['budget'].sum())

# average math score
average_math_score = round(school_data_complete['math_score'].mean(), 6)

# average reading score
average_reading_score = round(school_data_complete['reading_score'].mean(), 6)

# percentage of students with passing math score (70+)
plus70_math = school_data_complete.loc[school_data_complete["math_score"] >= 70]

percent_math_passing = round(((len(plus70_math) / total_students) * 100), 6)

# percentage of students with passing reading score (70+)
plus70_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70]

percent_reading_passing = round(((len(plus70_reading) / total_students) * 100), 6)

# percentage of students who passed math *and* reading (% Overall Passing)
plus70_overall = school_data_complete.loc[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)]

percent_overall_passing = round(((len(plus70_overall) / total_students) * 100), 6)
print(percent_overall_passing)

65.172326


In [3]:
# dataframe for district summary
# help from Stack Overflow
district_summary = pd.DataFrame({
    "Total Schools" : total_schools,
    "Total Students" : total_students_formatted,
    "Total Budget" : total_budget,
    "Average Math Score" : average_math_score,
    "Average Reading Score" : average_reading_score,
    "% Passing Math" : percent_math_passing,
    "% Passing Reading" : percent_reading_passing,
    "% Passing Overall" : percent_overall_passing}, index=[0])

# print district summary
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


School Summary

In [4]:
# initiate dataframe for school summary
# school name, school type, total students, total budget can be pulled from school_data df
school_summary = school_data.loc[:, ["school_name", "type", "size", "budget"]]

# rename columns
school_summary = school_summary.rename(columns={
    "school_name": "School Name", 
    "type": "School Type", 
    "size": "Total Students", 
    "budget": "Total School Budget"})

# reset index
school_summary = school_summary.set_index("School Name")

# sort schools alphabetically
school_summary = school_summary.sort_values(by="School Name")

# instantiate remaining columns of summary
# because we want 'Per Student Budget' to be formatted as currency, we need it to contain str objects - the rest can hold floats
school_summary['Per Student Budget'] = "0"
school_summary[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']] = 0
# school_summary.head()

In [5]:
# grab budget and student totals for each school
perSchool_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
perSchool_counts = school_data_complete["school_name"].value_counts()
# calculate per capita spending
perCapita_budget = perSchool_budget / perSchool_counts
school_summary['Per Student Budget'] = perCapita_budget

# average math and reading scores
averageMath_score = school_data_complete.groupby(["school_name"]).mean()["math_score"]
averageReading_score = school_data_complete.groupby(["school_name"]).mean()["reading_score"]

school_summary['Average Math Score'] = averageMath_score
school_summary['Average Reading Score'] = averageReading_score

# percent passing math, reading, and overall
# capture all students whose math/reading score >= 70 and sort them by school
# (https://stackoverflow.com/questions/36921951/truth-value-of-a-series-is-ambiguous-use-a-empty-a-bool-a-item-a-any-o)
plus70_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby(["school_name"])
plus70_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby(["school_name"])
plus70_overall = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)].groupby(["school_name"])

# calculate percent passing
perMath_passing = (plus70_math.math_score.size() / school_summary["Total Students"]) * 100
perReading_passing = (plus70_reading.reading_score.size() / school_summary["Total Students"]) * 100
perOverall_passing = (plus70_overall.math_score.size() / school_summary["Total Students"]) * 100

# store calculations in appropriate columns
school_summary['% Passing Math'] = perMath_passing
school_summary['% Passing Reading'] = perReading_passing
school_summary['% Overall Passing'] = perOverall_passing

# Total School Budget and Per Student Budget as currency
# help from Stack Overflow (https://stackoverflow.com/questions/35019156/pandas-format-column-as-currency)
school_summary['Total School Budget'] = school_summary['Total School Budget'].apply(lambda x: "${:,.2f}".format(x))
school_summary['Per Student Budget'] = school_summary['Per Student Budget'].apply(lambda y: "${:,.2f}".format(y))

# print school summary
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


Top Performing Schools (By % Overall Passing)

In [6]:
# top performing schools - sort and display the top 5 schools by % overall passing
topPerforming_summary = school_summary.sort_values(by='% Overall Passing', ascending=False)
topPerforming_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


Bottom Performing Schools (By % Overall Passing)

In [7]:
# bottom performing schools - sort and display the bottom 5 schools by % overall passing
bottomPerforming_summary = school_summary.sort_values(by='% Overall Passing')
bottomPerforming_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


Average Math Scores by Grade

In [8]:
# group students by grade and school name
ninth_grade = school_data_complete[school_data_complete["grade"] == "9th"].groupby(['school_name'])
tenth_grade = school_data_complete[school_data_complete["grade"] == "10th"].groupby(['school_name'])
eleventh_grade = school_data_complete[school_data_complete["grade"] == "11th"].groupby(['school_name'])
twelfth_grade = school_data_complete[school_data_complete["grade"] == "12th"].groupby(['school_name'])

# calculate average math scores
ninth_math = ninth_grade.math_score.mean()
tenth_math = tenth_grade.math_score.mean()
eleventh_math = eleventh_grade.math_score.mean()
twelfth_math = twelfth_grade.math_score.mean()

# create dataframe for average math scores by grade
mathScores_by_grade = pd.DataFrame({
    "School Name": school_summary.index,
    "9th": ninth_math,
    "10th": tenth_math,
    "11th": eleventh_math,
    "12th": twelfth_math
})

# set dataframe index
mathScores_by_grade = mathScores_by_grade.set_index("School Name", drop=True)

# print summary
mathScores_by_grade

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


Average Reading Scores by Grade

In [9]:
# calculate average reading scores
ninth_reading = ninth_grade.reading_score.mean()
tenth_reading = tenth_grade.reading_score.mean()
eleventh_reading = eleventh_grade.reading_score.mean()
twelfth_reading = twelfth_grade.reading_score.mean()

# create dataframe for average reading scores by grade
readingScores_by_grade = pd.DataFrame({
    "School Name": school_summary.index,
    "9th": ninth_reading,
    "10th": tenth_reading,
    "11th": eleventh_reading,
    "12th": twelfth_reading
})

# clean dataframe
readingScores_by_grade = readingScores_by_grade.set_index("School Name", drop=True)

# print summary
readingScores_by_grade

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


Scores by School Spending

In [10]:
# convert school_summary["Per Student Budget"] back to float so we can perform calculations 
# (https://stackoverflow.com/questions/32464280/converting-currency-with-to-numbers-in-python-pandas)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].replace('[\$,]', '', regex=True).astype(float)

# group schools by "Per Student Budget"
lessThan585 = school_summary.groupby(school_summary["Per Student Budget"] < 585)
between585_630 = school_summary.groupby((school_summary["Per Student Budget"] >= 585) & (school_summary["Per Student Budget"] < 630))
between630_645 = school_summary.groupby((school_summary["Per Student Budget"] >= 630) & (school_summary["Per Student Budget"] < 645))
greaterThan645 = school_summary.groupby(school_summary["Per Student Budget"] >= 645)

# average math score
lessThan585_math = lessThan585["Average Math Score"].mean()
between585_630_math = between585_630["Average Math Score"].mean()
between630_645_math = between630_645["Average Math Score"].mean()
greaterThan645_math = greaterThan645["Average Math Score"].mean()

# average reading score
lessThan585_reading = lessThan585["Average Reading Score"].mean()
between585_630_reading = between585_630["Average Reading Score"].mean()
between630_645_reading = between630_645["Average Reading Score"].mean()
greaterThan645_reading = greaterThan645["Average Reading Score"].mean()

# percent passing math
lessThan585_passingMath = lessThan585["% Passing Math"].mean()
between585_630_passingMath = between585_630["% Passing Math"].mean()
between630_645_passingMath = between630_645["% Passing Math"].mean()
greaterThan645_passingMath = greaterThan645["% Passing Math"].mean()

# percent passing reading
lessThan585_passingReading = lessThan585["% Passing Reading"].mean()
between585_630_passingReading = between585_630["% Passing Reading"].mean()
between630_645_passingReading = between630_645["% Passing Reading"].mean()
greaterThan645_passingReading = greaterThan645["% Passing Reading"].mean()

# percent passing overall
lessThan585_passingOverall = lessThan585["% Overall Passing"].mean()
between585_630_passingOverall = between585_630["% Overall Passing"].mean()
between630_645_passingOverall = between630_645["% Overall Passing"].mean()
greaterThan645_passingOverall = greaterThan645["% Overall Passing"].mean()

# create dataframe and set index
scores_schoolSpending = pd.DataFrame({
    "Spending Ranges (Per Student)": ["<$585", "$585-$630", "$630-$645", "$645-$680"],
    "Average Math Score": [lessThan585_math[1], between585_630_math[1], between630_645_math[1], greaterThan645_math[1]],
    "Average Reading Score": [lessThan585_reading[1], between585_630_reading[1], between630_645_reading[1], greaterThan645_reading[1]],
    "% Passing Math": [lessThan585_passingMath[1], between585_630_passingMath[1], between630_645_passingMath[1], greaterThan645_passingMath[1]],
    "% Passing Reading": [lessThan585_passingReading[1], between585_630_passingReading[1], between630_645_passingReading[1], greaterThan645_passingReading[1]],
    "% Overall Passing": [lessThan585_passingOverall[1], between585_630_passingOverall[1], between630_645_passingOverall[1], greaterThan645_passingOverall[1]]
})

scores_schoolSpending = scores_schoolSpending.set_index("Spending Ranges (Per Student)", drop=True)

# print summary
print(scores_schoolSpending)

                               Average Math Score  Average Reading Score  \
Spending Ranges (Per Student)                                              
<$585                                   83.455399              83.933814   
$585-$630                               81.899826              83.155286   
$630-$645                               78.518855              81.624473   
$645-$680                               76.997210              81.027843   

                               % Passing Math  % Passing Reading  \
Spending Ranges (Per Student)                                      
<$585                               93.460096          96.610877   
$585-$630                           87.133538          92.718205   
$630-$645                           73.484209          84.391793   
$645-$680                           66.164813          81.133951   

                               % Overall Passing  
Spending Ranges (Per Student)                     
<$585                           

Scores by School Size

In [11]:
# group schools by size
smallSchools = school_summary.groupby(school_summary["Total Students"] < 1000)
medSchools = school_summary.groupby((school_summary["Total Students"] >= 1000) & (school_summary["Total Students"] < 2000))
largeSchools = school_summary.groupby((school_summary["Total Students"] >= 2000) & (school_summary["Total Students"] < 5000))

# average math score
smallSchools_math = smallSchools["Average Math Score"].mean()
medSchools_math = medSchools["Average Math Score"].mean()
largeSchools_math = largeSchools["Average Math Score"].mean()

# average reading score
smallSchools_reading = smallSchools["Average Reading Score"].mean()
medSchools_reading = medSchools["Average Reading Score"].mean()
largeSchools_reading = largeSchools["Average Reading Score"].mean()

# percent passing math
smallSchools_passingMath = smallSchools["% Passing Math"].mean()
medSchools_passingMath = medSchools["% Passing Math"].mean()
largeSchools_passingMath = largeSchools["% Passing Math"].mean()

# percent passing reading
smallSchools_passingReading = smallSchools["% Passing Reading"].mean()
medSchools_passingReading = medSchools["% Passing Reading"].mean()
largeSchools_passingReading = largeSchools["% Passing Reading"].mean()

# percent passing overall
smallSchools_passingOverall = smallSchools["% Overall Passing"].mean()
medSchools_passingOverall = medSchools["% Overall Passing"].mean()
largeSchools_passingOverall = largeSchools["% Overall Passing"].mean()

# create dataframe and set index
scores_schoolSize = pd.DataFrame({
    "School Size": ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"],
    "Average Math Score": [smallSchools_math[1], medSchools_math[1], largeSchools_math[1]],
    "Average Reading Score": [smallSchools_reading[1], medSchools_reading[1], largeSchools_reading[1]],
    "% Passing Math": [smallSchools_passingMath[1], medSchools_passingMath[1], largeSchools_passingMath[1]],
    "% Passing Reading": [smallSchools_passingReading[1], medSchools_passingReading[1], largeSchools_passingReading[1]],
    "% Overall Passing": [smallSchools_passingOverall[1], medSchools_passingOverall[1], largeSchools_passingOverall[1]]
})

scores_schoolSize = scores_schoolSize.set_index("School Size", drop=True)

# print summary
print(scores_schoolSize)

                    Average Math Score  Average Reading Score  % Passing Math  \
School Size                                                                     
Small (<1000)                83.821598              83.929843       93.550225   
Medium (1000-2000)           83.374684              83.864438       93.599695   
Large (2000-5000)            77.746417              81.344493       69.963361   

                    % Passing Reading  % Overall Passing  
School Size                                               
Small (<1000)               96.099437          89.883853  
Medium (1000-2000)          96.790680          90.621535  
Large (2000-5000)           82.766634          58.286003  


Scores by School Type

In [12]:
# average math/reading scores, passing percentages by school type 
averageMath_score = school_summary.groupby(["School Type"]).mean()["Average Math Score"]
averageReading_score = school_summary.groupby(["School Type"]).mean()["Average Reading Score"]

perMath_passing = school_summary.groupby(["School Type"]).mean()["% Passing Math"]
perReading_passing = school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
perOverall_passing = school_summary.groupby(["School Type"]).mean()["% Overall Passing"]

# create dataframe and set index
scores_schoolType = pd.DataFrame({
    "School Type": ["Charter", "District"],
    "Average Math Score": averageMath_score,
    "Average Reading Score": averageReading_score,
    "% Passing Math": perMath_passing,
    "% Passing Reading": perReading_passing,
    "% Overall Passing": perOverall_passing
})

scores_schoolType = scores_schoolType.set_index("School Type", drop=True)

print(scores_schoolType)

             Average Math Score  Average Reading Score  % Passing Math  \
School Type                                                              
Charter               83.473852              83.896421       93.620830   
District              76.956733              80.966636       66.548453   

             % Passing Reading  % Overall Passing  
School Type                                        
Charter              96.586489          90.432244  
District             80.799062          53.672208  
